This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/robot.html).  I recommend having both windows open, side-by-side!

In [ ]:
from itertools import product

import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    ContactVisualizer,
    DiagramBuilder,
    ExternallyAppliedSpatialForce,
    LeafSystem,
    List,
    MeshcatVisualizer,
    ModelVisualizer,
    Parser,
    Simulator,
    SpatialForce,
    StartMeshcat,
    Value,
)

from manipulation import ConfigureParser, FindResource, running_as_notebook

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

# Rubik's Cube (2x2)

TODO(russt): Use the quaternion ball joint when it's available:
https://github.com/RobotLocomotion/drake/issues/12404
(rotating around y can hit a singularity)

In [ ]:
cube_file = "package://manipulation/rubiks_cube_2_by_2.sdf"

meshcat.Delete()

visualizer = ModelVisualizer(meshcat=meshcat)
ConfigureParser(visualizer.parser())
visualizer.parser().AddModelsFromUrl(cube_file)

visualizer.Run(loop_once=not running_as_notebook)

meshcat.DeleteAddedControls()

In [ ]:
rotate_about = "x"
# rotate_about = 'y'
# rotate_about = 'z'

meshcat.Delete()
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0.001)
parser = Parser(plant)
ConfigureParser(parser)
parser.AddModelsFromUrl(cube_file)

plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("center"))
plant.Finalize()

visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
ContactVisualizer.AddToBuilder(builder, plant, meshcat)
# logger = LogVectorOutput(plant.get_state_output_port(), builder)


class CubePusher(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)
        forces_cls = Value[List[ExternallyAppliedSpatialForce]]
        self.DeclareAbstractOutputPort(
            "applied_force", lambda: forces_cls(), self.CalcOutput
        )

    def CalcOutput(self, context, output):
        forces = []
        for x, y, z in product([0, 1], repeat=3):
            force = ExternallyAppliedSpatialForce()
            force.body_index = plant.GetBodyByName(f"box_{x}_{y}_{z}").index()
            # shift from [0, 1] to [-1, 1]
            x = 2 * x - 1
            y = 2 * y - 1
            z = 2 * z - 1
            force.p_BoBq_B = -0.0125 * np.array([x, y, z])  # world 0, 0, 0
            if rotate_about == "x":
                force.F_Bq_W = SpatialForce(
                    tau=-0.2 * np.array([1 if x < 0 else -1, 0, 0]),
                    f=[0, 0, 0],
                )
            elif rotate_about == "y":
                force.F_Bq_W = SpatialForce(
                    tau=0.2 * np.array([0, 1 if y > 0 else -1, 0]), f=[0, 0, 0]
                )
            else:
                force.F_Bq_W = SpatialForce(
                    tau=0.2 * np.array([0, 0, 1 if z > 0 else -1]), f=[0, 0, 0]
                )
        forces.append(force)
        output.set_value(forces)


pusher = builder.AddSystem(CubePusher())
builder.Connect(pusher.get_output_port(), plant.get_applied_spatial_force_input_port())

diagram = builder.Build()

simulator = Simulator(diagram)
context = simulator.get_context()

if running_as_notebook:
    simulator.set_target_realtime_rate(1.0)
    simulator.AdvanceTo(10)
else:
    simulator.AdvanceTo(0.1)

# Rubik's Cube (3x3)

In [ ]:
def generate_3_by_3():
    filename = FindResource("models/rubiks_cube.sdf")
    box_size = 0.025
    box_mass = 0.03
    inertia = box_mass * (box_size**2) / 6
    inertia = inertia * 100  # scale inertia to help numerics

    with open(filename, "w") as f:
        f.write(
            """
<?xml version="1.0"?>
<!-- Autogenerated by rubiks_cube.ipynb -->
<sdf version="1.7">
  <model name="rubiks_cube">
    <link name="center"/>
"""
        )

        # TODO(russt): I could reduce the dofs for the center cubes, since
        # box_m1_0_0 and box_1_0_0, must have their relative poses
        # constrained up to a single-axis rotation, etc.

        # TODO(russt): Currently I'm adding some faces on the inside of the
        # internal blocks (e.g. the x face when x==0). They are inelegant and
        # mildly inefficiently, but won't be visible.
        for x, y, z in product([-1, 0, 1], repeat=3):
            if x == 0 and y == 0 and z == 0:
                # No cube in the very center
                continue
            sx = x if x >= 0 else "m1"
            sy = y if y >= 0 else "m1"
            sz = z if z >= 0 else "m1"
            suffix = f"_{sx}_{sy}_{sz}"
            x_color = [0, 0, 1] if x < 0 else [0, 1, 0]
            y_color = [1, 0.5, 0] if y < 0 else [1, 0, 0]
            z_color = [1, 1, 0] if z < 0 else [1, 1, 1]
            f.write(
                f"""
    <link name="box{suffix}">
      <pose>{box_size*x} {box_size*y} {box_size*z} 0 0 0</pose>
      <inertial>
        <mass>0.03</mass>
        <inertia>
          <ixx>0.0003125</ixx>
          <ixy>0.0</ixy>
          <ixz>0.0</ixz>
          <iyy>0.0003125</iyy>
          <iyz>0.0</iyz>
          <izz>0.0003125</izz>
        </inertia>
      </inertial>
      <collision name="collision">
        <geometry>
          <box>
            <size>{box_size} {box_size} {box_size}</size>
          </box>
        </geometry>
      </collision>
      <visual name="black">
        <geometry>
          <box>
            <size>{box_size} {box_size} {box_size}</size>
          </box>
        </geometry>
        <material>
          <diffuse>0 0 0 1</diffuse>
        </material>
      </visual>
      <visual name="x">
        <pose>{x*box_size/2} 0 0 0 0 0</pose>
        <geometry>
          <box>
            <size>0.0002 {box_size*4/5} {box_size*4/5}</size>
          </box>
        </geometry>
        <material>
          <diffuse>{x_color[0]} {x_color[1]} {x_color[2]} 1</diffuse>
        </material>
      </visual>
      <visual name="y">
        <pose>0 {y*box_size/2} 0 0 0 0</pose>
        <geometry>
          <box>
            <size>{box_size*4/5} 0.0002 {box_size*4/5}</size>
          </box>
        </geometry>
        <material>
          <diffuse>{y_color[0]} {y_color[1]} {y_color[2]} 1</diffuse>
        </material>
      </visual>
      <visual name="z">
        <pose>0 0 {z*box_size/2} 0 0 0</pose>
        <geometry>
          <box>
            <size>{box_size*4/5} {box_size*4/5} 0.0002</size>
          </box>
        </geometry>
        <material>
          <diffuse>{z_color[0]} {z_color[1]} {z_color[2]} 1</diffuse>
        </material>
      </visual>
    </link>      
    <joint name="ball{suffix}" type="ball">
        <pose>{-x*box_size} {-y*box_size} {-z*box_size} 0 0 0</pose> <!-- in child frame -->
        <parent>center</parent>
        <child>box{suffix}</child>
        <axis>
            <dynamics><damping>0.1</damping></dynamics>
            <limit><effort>0</effort></limit>
        </axis>
    </joint>
"""
            )

        f.write(
            """
  </model>
</sdf>
"""
        )


if running_as_notebook:
    generate_3_by_3()

In [ ]:
cube_file = "package://manipulation/rubiks_cube.sdf"

meshcat.Delete()

visualizer = ModelVisualizer(meshcat=meshcat)
ConfigureParser(visualizer.parser())
visualizer.parser().AddModelsFromUrl(cube_file)

visualizer.Run(loop_once=not running_as_notebook)

meshcat.DeleteAddedControls()